### Fase 1: Análisis Exploratorio de datos

#### **Elección del Conjunto de datos**

---

#### **Planteamiento de la problemáticas**

---

#### **Analisis de la serie de tiempo**

---

#### **Elección de Variables Exógenas**

---

#### **Elección de Métrica de error**

---